<a href="https://colab.research.google.com/github/vlademirribeiro/challenge_alura_telecon_2/blob/main/telecom2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# Bibliotecas do Skit Learn
from sklearn.model_selection import train_test_split #Utilizada para separar dados pra treino e teste
from sklearn.preprocessing import StandardScaler #Utilizada para fazer a padronização dos dados
from sklearn.preprocessing import LabelEncoder #Utilizada para fazer o OneHotEncoding
from sklearn.metrics import accuracy_score #Utilizada para avaliar a acurácia do modelo preditivo
from sklearn.neighbors import KNeighborsClassifier #Nosso Algoritmo para criação do modelo
from imblearn import under_sampling, over_sampling #Utilizada para fazer o balanceamento de dados
from imblearn.over_sampling import SMOTE #Utilizada para fazer o balanceamento de dados
from sklearn.preprocessing import StandardScaler # Utilizado para fazer a normalização dos dados
from sklearn.preprocessing import MinMaxScaler # Utilizado para fazer a normalização dos dados
from sklearn.preprocessing import LabelEncoder # Utilizado para fazer o OneHotEncoding
from sklearn.linear_model import LinearRegression # Algoritmo de Regressão Linear
from sklearn.metrics import r2_score # Utilizado para medir a acuracia do modelo preditivo
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.svm import SVC
from sklearn.naive_bayes import CategoricalNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import requests
import json

url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/refs/heads/main/TelecomX_Data.json"
response = requests.get(url)
data = response.json()


In [ ]:
# convertendo para dataframe
df = pd.DataFrame(data)
df.head()

In [ ]:
dados_normalizados = pd.json_normalize(data)
dados_normalizados.head()

In [ ]:
dados_normalizados.shape

In [ ]:
dados_normalizados.info()

In [ ]:
print(dados_normalizados.columns.tolist())

In [ ]:
dados_normalizados.drop(['customerID'], axis=1, inplace=True)

In [ ]:
cols_to_fix = [
    'internet.OnlineSecurity', 'internet.OnlineBackup', 'internet.DeviceProtection',
    'internet.TechSupport', 'internet.StreamingTV', 'internet.StreamingMovies'
]

for col in cols_to_fix:
    dados_normalizados[col] = dados_normalizados[col].replace('No internet service', 'No')

In [ ]:
dados_normalizados.head()

In [ ]:
dados_normalizados.isna().sum()

In [ ]:
dados_normalizados.describe()


In [ ]:
dados_normalizados.nunique()

In [ ]:
variaveis_numericas = []

for i in dados_normalizados.columns[0:22].tolist():

    # Verifica se o tipo de dado da coluna atual é 'int64' ou 'float64' (números inteiros ou reais)
    if dados_normalizados.dtypes[i] == 'int64' or dados_normalizados.dtypes[i] == 'float64':

        # Imprime o nome da coluna e o tipo de dado dela
        print(i, ':', dados_normalizados.dtypes[i])

        # Adiciona o nome da coluna à lista de variáveis numéricas
        variaveis_numericas.append(i)

In [ ]:
# Define o tamanho da figura em polegadas: largura = 14, altura = 5
plt.rcParams["figure.figsize"] = [14.00, 5.00]

# Garante que os elementos do gráfico se ajustem automaticamente ao layout
plt.rcParams["figure.autolayout"] = True

# Como temos 5 variáveis, podemos criar uma grade de 1 linha e 5 colunas
f, axes = plt.subplots(1, 3)  # 1 linha e 5 colunas

# Inicializa a posição dos gráficos
coluna = 0

# Percorre todas as variáveis numéricas selecionadas
for i in variaveis_numericas[:5]:  # Seleciona as primeiras 5 variáveis numéricas
    # Cria um boxplot para a variável i na posição [0][coluna] da grade
    sns.boxplot(data=dados_normalizados, y=i, ax=axes[coluna])

    # Avança para a próxima coluna
    coluna += 1

# Exibe todos os gráficos gerados
plt.show()


In [ ]:
# Lista para armazenar as variáveis categóricas
variaveis_categoricas = []

# Itera sobre as colunas de 0 até a 47 (isso seleciona as primeiras 48 colunas)
for i in dados_normalizados.columns[0:23]:  # Corrigindo a iteração para cada coluna
    if dados_normalizados.dtypes[i] == 'object' or dados_normalizados.dtypes[i] == 'category':
        # Imprime o nome da variável e o seu tipo
        print(i, ':', dados_normalizados.dtypes[i])
        # Adiciona a variável à lista de variáveis categóricas
        variaveis_categoricas.append(i)


# Exibe as variáveis categóricas identificadas
print("Variáveis categóricas:",len(variaveis_categoricas), variaveis_categoricas)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import math

def plot_categoricas(dados, variaveis, ncols=4, figsize=(15, 15)):
    """
    Plota gráficos de contagem para variáveis categóricas.

    :param dados: DataFrame com os dados normalizados
    :param variaveis: lista de colunas categóricas
    :param ncols: número de colunas na grade de gráficos
    :param figsize: tamanho da figura
    """
    n_plots = len(variaveis)
    nrows = math.ceil(n_plots / ncols)

    fig, axes = plt.subplots(nrows, ncols, figsize=figsize)
    axes = axes.flatten()  # transforma em lista 1D para simplificar

    for ax, var in zip(axes, variaveis):
        sns.countplot(data=dados, x=var, ax=ax)
        ax.set_title(var)

    # Remove eixos extras (se sobrar espaço na grade)
    for ax in axes[n_plots:]:
        ax.remove()

    plt.tight_layout()
    plt.show()


# Exemplo de uso
# plot_categoricas(dados_normalizados, variaveis_categoricas)



'''
# Com este comando iremos exibir todos gráficos de todas colunas de uma vez só para facilitar nossa análise.

# Aqui definimos o tamanho da tela para exibição dos gráficos
plt.rcParams["figure.figsize"] = [15.00, 15.00]
plt.rcParams["figure.autolayout"] = True

# Aqui definimos em quantas linhas e colunas queremos exibir os gráficos
# Usando 5 linhas e 4 colunas para acomodar 17 gráficos
f, axes = plt.subplots(4, 4)  # 5 linhas e 4 colunas (total de 20 gráficos)

# Inicializa a posição dos gráficos
linha = 0
coluna = 0

# Itera sobre as variáveis categóricas
for i in variaveis_categoricas:
    # Verifica se a posição de coluna excedeu o número de colunas (4)
    sns.countplot(data=dados_normalizados, x=i, ax=axes[linha][coluna])

    coluna += 1
    if coluna == 4:  # Quando atingir a 4ª coluna, vai para a próxima linha
        linha += 1
        coluna = 0

# Ajusta o layout para que todos os gráficos sejam exibidos corretamente
plt.tight_layout()

# Exibe os gráficos
plt.show()
'''

In [ ]:
plt.rcParams["figure.figsize"] = [15.00, 5.00]
plt.rcParams["figure.autolayout"] = True

# Aqui definimos em quantas linhas e colunas queremos exibir os gráficos
f, axes = plt.subplots(2, 3) #4 linhas e 3 colunas

linha = 0
coluna = 0

for i in variaveis_numericas:
    sns.histplot(data = dados_normalizados, x=i, ax=axes[linha][coluna])
    coluna += 1
    if coluna == 3:
        linha += 1
        coluna = 0

plt.show()


In [ ]:
dados_normalizados.head()

In [ ]:
variaveis_categoricas

In [ ]:
mapa_binario = {'Yes': 1, 'No': 0, 'Female': 0, 'Male': 1}
colunas_para_mapear = ['Churn', 'customer.gender', 'customer.Partner', 'customer.Dependents', 'phone.PhoneService', 'account.PaperlessBilling']

for col in colunas_para_mapear:
    # Apenas mapeia se a coluna ainda for do tipo 'object'
    if dados_normalizados[col].dtype == 'object':
        dados_normalizados[col] = dados_normalizados[col].map(mapa_binario)

# Identifica colunas multinomiais que ainda são 'object'
colunas_multinomiais = dados_normalizados.select_dtypes(include=['object']).columns

# Aplica One-Hot Encoding (get_dummies)
dados = pd.get_dummies(dados_normalizados, columns=colunas_multinomiais, drop_first=True)

print("Dados após One-Hot Encoding:")
dados.head()

In [ ]:
# Vamos aplicar a normalização

# Selecionando apenas colunas numéricas (exceto a variável target, se já estiver separada)
colunas_numericas = dados_normalizados.select_dtypes(include=['int64', 'float64']).columns

# Inicializando o scaler
scaler = MinMaxScaler()

# Aplicando a normalização
dados_normalizados[colunas_numericas] = scaler.fit_transform(dados[colunas_numericas])

# Exibindo os dados normalizados
dados.head()


In [ ]:
corr = dados_normalizados.corr(numeric_only=True)

# Cria uma máscara para ocultar a parte superior da matriz (duplicada)
mask = np.triu(np.ones_like(corr, dtype=bool))

# Configuração da figura
fig, ax = plt.subplots(figsize=(8, 10))

# Heatmap estilizado
sns.heatmap(
    np.round(corr, 2),
    mask=mask,               # Máscara triangular
    cmap='RdBu',             # Mapa de cores contrastante
    vmax=1, vmin=-1, center=0,
    square=True,
    linewidths=.5,
    annot=True,              # Mostrar valores
    annot_kws={"size": 10},   # Tamanho do texto
    cbar_kws={"shrink": .5}   # Barra de cores menor
)

# Ajusta rótulos
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(rotation=0, fontsize=12)

# Título
plt.title('Matriz de Correlação das Variáveis', fontsize=18, pad=20)

plt.show()


In [ ]:
plt.figure(figsize=(8, 10))

# Calcula a matriz de correlação do DataFrame e filtra para mostrar apenas as correlações com 'Churn'
# Ordena os valores para visualizar facilmente os fatores mais correlacionados (positiva ou negativamente)
heatmap = sns.heatmap(
    dados_normalizados.corr(numeric_only=True)[['Churn']].sort_values(by='Churn', ascending=False),
    vmin=-1, vmax=1, annot=True, cmap='BrBG'
)

# Título do gráfico
heatmap.set_title('Features Correlacionadas com Churn', fontdict={'fontsize': 18}, pad=16)
plt.show()


In [ ]:
dados_normalizados.info()

In [ ]:
dados_normalizados.head()


In [ ]:
# 1. Separar X e y do dataframe original (antes do SMOTE)
X = dados_normalizados.drop('Churn', axis=1)
y = dados_normalizados['Churn']


In [ ]:
# 2. Dividir em dados de treino e teste PRIMEIRO
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=40)

In [ ]:
# Exibe a distribuição normalizada da variável 'Churn' em percentual
churn_distribution = dados_normalizados['Churn'].value_counts(normalize=True) * 100

# Exibe os resultados com duas casas decimais
print(churn_distribution.apply(lambda x: f'{x:.2f}%'))

In [ ]:
print("Quantidade de NaN em y:", y.isna().sum())
print("Total de linhas em y:", len(y))
print("Proporção de NaN em y:", y.isna().mean())

print("\nVerificando X:")
print(X.isna().sum())  # mostra colunas com NaN


In [ ]:
print("Shape de X:", X.shape)
print("Shape de y:", y.shape)

# Se estiver usando treino/teste
print("Shape de X_treino:", X_treino.shape)
print("Shape de y_treino:", y_treino.shape)


In [ ]:
# Para classificação binária
y = y.fillna(y.mode()[0])  # preenche com a moda (valor mais frequente)
y = y.astype(int)           # garante que seja inteiro


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# =========================
# 1️⃣ Função de validação
# =========================
def validar_dados(X, y, etapa="pré-processamento", parar_em_erro=True):
    """
    Verifica se X e y têm dados válidos.
    Mostra alertas se estiverem vazios ou com NaN.
    """
    print(f"\n🔎 Validando dados após etapa: {etapa}")
    print(f"➡ Shape de X: {X.shape}")
    print(f"➡ Shape de y: {y.shape}")

    if X.shape[0] == 0 or y.shape[0] == 0:
        msg = f"❌ Dataset vazio após {etapa}. Verifique filtros ou origem dos dados."
        if parar_em_erro:
            raise ValueError(msg)
        else:
            print(msg)
            return False

    if y.isna().sum() > 0:
        print(f"⚠️ Atenção: y contém {y.isna().sum()} valores NaN.")

    na_x = X.isna().sum()
    if na_x.sum() > 0:
        print("⚠️ Atenção: X contém valores NaN nas colunas abaixo:")
        print(na_x[na_x > 0])

    print("✅ Dados válidos!")
    return True

# =========================
# 2️⃣ Exemplo de dados
# =========================
# Substitua pelo seu dataset real
# df = pd.read_csv("seus_dados.csv")
# X = df.drop("target", axis=1)
# y = df["target"]

X = pd.DataFrame({
    "A": [1, 2, 3, None, 5],
    "B": [5, 4, None, 2, 1]
})
y = pd.Series([0, 1, None, 0, 1])

# =========================
# 3️⃣ Tratar NaN
# =========================
X = X.fillna(X.median())
y = y.fillna(y.median())

# =========================
# 4️⃣ Validar dados
# =========================
validar_dados(X, y, etapa="após tratamento de NaN")

# =========================
# 5️⃣ Divisão treino/teste
# =========================
X_treino, X_teste, y_treino, y_teste = train_test_split(
    X, y, test_size=0.2, random_state=42
)
validar_dados(X_treino, y_treino, etapa="train_test_split")

# =========================
# 6️⃣ Aplicar SMOTE
# =========================
smote = SMOTE(random_state=42)
X_treino_res, y_treino_res = smote.fit_resample(X_treino, y_treino)

print(f"\n✅ SMOTE aplicado com sucesso! Novo shape treino: {X_treino_res.shape}")


In [ ]:
import pandas as pd
from imblearn.over_sampling import SMOTE

# Remove valores NaN em y (e nas linhas correspondentes de X)
mask = ~pd.isna(y)   # mantém apenas onde y NÃO é NaN
X = X.loc[mask]
y = y.loc[mask]

# (Opcional) Se preferir preencher em vez de remover, use:
 #y = y.fillna(y.median())

# Definir seed
seed = 100

# Balanceamento com SMOTE
smote_bal = SMOTE(random_state=seed)
X_treino_res, y_treino_res = smote_bal.fit_resample(X, y)

print("✅ Dados balanceados com SMOTE sem valores NaN!")


In [ ]:
print(pd.isna(y).sum())

In [ ]:
mask = pd.isna(y)
X = X[mask]
y = y[mask]


In [ ]:
y = y.fillna(y.median())   # ou y.mean(), ou outro valor adequado


In [ ]:
# 3. Aplicar o SMOTE APENAS nos dados de treino
seed = 100
smote_bal = SMOTE(random_state=seed)
X_treino_res, y_treino_res = smote_bal.fit_resample(X_treino, y_treino)

In [ ]:
print("\nDistribuição das classes após o balanceamento:")
print(y.value_counts())

# Exibe o número de amostras antes e depois
print(f"\nNúmero de amostras antes do balanceamento: {len(y)}")
print(f"Número de amostras após o balanceamento: {len(y)}")

In [ ]:

# Range de valores de k que iremos testar
kVals = range(3, 10, 2)

# Lista vazia para receber as acurácias
acuracias = []
start = time.time()

# Loop para treinar e avaliar o modelo para cada valor de k
for k in kVals:
    # Treinando o modelo KNN com cada valor de k
    modeloKNN = KNeighborsClassifier(n_neighbors=k)
    modeloKNN.fit(X_treino_res, y_treino_res)

    # Avaliando o modelo e atualizando a lista de acurácias
    score = modeloKNN.score(X_teste, y_teste)
    print(f"Com valor de k = {k}, a acurácia é = {score * 100:.2f}%")
    acuracias.append(score)

# Obtendo o valor de k que apresentou a maior acurácia
i = np.argmax(acuracias)
print(f"\nO valor de k = {kVals[i]} alcançou a mais alta acurácia de {acuracias[i] * 100:.2f}% nos dados de validação!")

end = time.time()
print('Tempo de Treinamento do Modelo:', end - start)

# Criando a versão final do modelo com o maior valor de k
modeloFinal = KNeighborsClassifier(n_neighbors=kVals[i])
modeloFinal.fit(X_treino, y_treino)

# Previsões com os dados de teste
previsoes = modeloFinal.predict(X_teste)

# Calculando a acurácia do modelo final
acuracia = accuracy_score(y_teste, previsoes)

# Exibindo a acurácia formatada em percentual
print(f'Acurácia do modelo final: {acuracia * 100:.2f}%')


In [ ]:
# Instanciar e treinar
lr = LogisticRegression(random_state=42)
lr.fit(X_treino_res, y_treino_res)

# Previsões
y_pred_lr = lr.predict(X_teste)
y_prob_lr = lr.predict_proba(X_teste)[:, 1]

# Avaliação
print("Regressão Logística")
print("Acurácia:", accuracy_score(y_teste, y_pred_lr))
print("ROC AUC:", roc_auc_score(y_teste, y_prob_lr))
print("Matriz de Confusão:\n", confusion_matrix(y_teste, y_pred_lr))
print(classification_report(y_teste, y_pred_lr))

In [ ]:

# Definição do modelo base
rf = RandomForestClassifier(random_state=42)

# Grade de parâmetros para testar
param_grid = {
    'n_estimators': [100, 200, 300],       # número de árvores
    'max_depth': [10, 20],        # profundidade máxima da árvore
    'min_samples_split': [2, 5, 10],       # mínimo de amostras para dividir um nó
    'min_samples_leaf': [1, 2, 4],         # mínimo de amostras por folha
    'max_features': ['sqrt', 'log2']       # como selecionar features em cada divisão
}

# Busca em grade com validação cruzada
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,                     # 5-fold cross validation
    scoring='roc_auc',        # métrica alvo
    n_jobs=-1,                 # usar todos os núcleos disponíveis
    verbose=2
)

# Treinamento
grid_search.fit(X_treino_res, y_treino_res)

print("Melhores parâmetros encontrados:")
print(grid_search.best_params_)
print("Melhor pontuação ROC AUC:", grid_search.best_score_)


In [ ]:
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("="*60)
print(" 🎯 MELHOR MODELO PARA ESTA ANÁLISE 🎯 ".center(60, " "))
print("="*60)
print("\n📌 Modelo escolhido: RandomForestClassifier")
print(f"🔹 Melhores parâmetros: {best_params}")
print(f"📊 Melhor ROC AUC: {best_score:.4f}")
print("\n✅ Este modelo apresentou a melhor performance entre os testados e será utilizado para as próximas etapas da análise.")
print("="*60)